In [17]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from torch import nn,optim
import torch

import algo
import attack
import scripts

import os
import matplotlib.pyplot as plt
import json

rand_seed=24
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)

In [58]:
# purchase
# load data with different seeds
data_seed = {}
for rand_seed in [1,3,13,24,42]:
    data_seed[rand_seed] =  {}
    
    data_seed[rand_seed]['x_target_train'] = np.load('data/rs'+str(rand_seed)+'_x_target_train.npy')
    data_seed[rand_seed]['y_target_train'] = np.load('data/rs'+str(rand_seed)+'_y_target_train.npy')
    data_seed[rand_seed]['x_target_test'] = np.load('data/rs'+str(rand_seed)+'_x_target_test.npy')
    data_seed[rand_seed]['y_target_test'] = np.load('data/rs'+str(rand_seed)+'_y_target_test.npy')
    data_seed[rand_seed]['n_classes'] = len(np.unique(data_seed[rand_seed]['y_target_train']))
    data_seed[rand_seed]['X_train_size'] = data_seed[rand_seed]['x_target_train'].shape[0]
    data_seed[rand_seed]['X_test_size ']= data_seed[rand_seed]['x_target_test'].shape[0]

def set_the_seed_and_data(seed):
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)
    random.seed(rand_seed)
    
    return data_seed[seed]['x_target_train'], data_seed[rand_seed]['y_target_train'], data_seed[rand_seed]['x_target_test'], data_seed[rand_seed]['y_target_test']

In [59]:
# attack models
from torch import nn

class Net_attack(nn.Module):

    def __init__(self, h_neurons, do, input_size):
        super(Net_attack, self).__init__()
        self.input_size = input_size
        self.h_neurons = h_neurons
        self.do = do
        self.fc1 = nn.Linear(input_size, h_neurons)
        self.fc2 = nn.Linear(h_neurons, 2)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(do)
        self.softmax = nn.Softmax(dim=1)   

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.softmax(x)
        return x

path = 'mia'
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if "best_mi_model" in file:
            ams[file] = Net_attack(h_neurons=64, do=0, input_size=200)
            ams[file] = torch.load(r+'/'+file)      

# MI on centralized 

In [ ]:
path = 'tm'
tms_params = {}
tms = {}
for r,d,f in os.walk(path):
    for file in f:
        if "target_model_params.json" in file:
            with open(path+'/'+file) as json_file:
                tms_params[file.replace('_params.json', '')] = json.load(json_file)
        if "target_model.npy" in file:
            tms[file.replace('.npy', '')] = np.load(path+'/'+file)
            
df = pd.DataFrame.from_dict(tms_params, orient='index')
df.shape            

In [ ]:
# attack every centralized target model
for file in tms_params:
    
    if 'attack_acc_mean' in tms_params[file]:
        continue
    # set random seed and load the data    
    rand_seed=int(file[file.find('rs')+2:file.find('_lr')])
    x_target_train, y_target_train, x_target_test, y_target_test = set_the_seed_and_data(rand_seed)
    
    #attack
    target_model = algo.LogisticRegression_DPSGD()
    target_model.theta = tms[file]
    params = tms_params[file]
    scripts.set_model_params(target_model, params)
    attack_dict = attack.test_mi_attack(ams, target_model, x_target_train, y_target_train, x_target_test, y_target_test)
    
    params.update(attack_dict)
    if 'attack_acc' in params:
        params.pop('attack_acc')
        params.pop('attack_pre')
        params.pop('attack_rec')
    #write a new parameters file with attack results
    with open('tm/'+file+'_params.json', 'w') as file:
        json.dump(params, file)
        

# Membership inference Federated Learning


In [60]:
path = 'fl/'
params = {}
results = {}
models = {}
for r,d,f in os.walk(path):
    for file in f:
        if "params.json" in file:
            with open(r+'/'+file) as json_file:
                params[r] = json.load(json_file)
        elif "results.csv" in file:

#             with open(r+'/'+file, 'r') as f_open:
#                 lines = f_open.readlines()
#                 if 'HEAD' in lines[0]:   
#                     for i,l in enumerate(lines):
#                         if 'attack_acc_mean' in l:
#                             index = i
#                             break
#                     with open(r+'/'+file, 'w') as f_write:
#                         f_write.writelines(lines[i:-1])

            results[r] = pd.read_csv(r+'/'+file)
            it = []
            client = []
            model_filenames = []
            if 'Unnamed: 0' in results[r].keys():
                for k in results[r]['Unnamed: 0']:
                    it.append(k[k.find('i')+1:k.find('_')])
                    client.append(k[k.find('_')+1:])
                    model_filenames.append(k+'.npy')
                results[r]['it'] = it
                results[r]['client'] = client
                results[r]['file_name'] = model_filenames
                results[r].pop('Unnamed: 0')
        elif '.npy' in file:
            if r not in models:
                models[r] = {}
#             print(r+'/'+file)
            models[r][file] = np.load(r+'/'+file)



In [61]:
selected_files = results.keys()
len(selected_files)

266

In [62]:
# attack every federated local/global model
for file in selected_files:
    
    if 'attack_acc_mean' in results[file].keys():
        continue
        
    # set random seed and load the data    
    rand_seed=int(file[file.find('rs')+2:file.find('_ncl')])
    x_target_train, y_target_train, x_target_test, y_target_test = set_the_seed_and_data(rand_seed)
    #set number of client for following split of the training data between clients
    number_of_clients = len(results[file]['client'].unique())-1
    data_per_client = int(x_target_train.shape[0]/number_of_clients)

    #attack
    attack_results = {}
    print(file)
    for tm in results[file]['file_name']:
        target_model = algo.LogisticRegression_DPSGD()
        target_model.theta = models[file][tm]
        tm_params = params[file]
        scripts.set_model_params(target_model, tm_params)
        if 'g' in tm:
            target_model.x = x_target_train
            target_model.y = y_target_train
        else:
            i = int(tm[tm.find('_c')+2:tm.find('.npy')])
            target_model.x = x_target_train[i*data_per_client:(i+1)*data_per_client]
            target_model.y = y_target_train[i*data_per_client:(i+1)*data_per_client]
        attack_dict = attack.test_mi_attack(ams, target_model, target_model.x, target_model.y, x_target_test, y_target_test)
        attack_results[tm] = attack_dict
    attack_df = pd.DataFrame.from_dict(attack_results, orient='index')
    result_df = results[file].set_index('file_name')
    new_df = pd.merge(result_df, attack_df, left_index=True, right_index=True)
    #save attack results in old results file
    new_df.to_csv(file+'/results.csv')
    results[file] = new_df

fl/rs42_ncl32_fiter20_lr0.01_iter200_reg0.0001_sgdDPTrue_eps10000_L20_C2
fl/rs42_ncl32_fiter15_lr0.01_iter300_reg0.0001_sgdDPTrue_eps10000_L20_C2


In [63]:
new_df

,train_acc,test_acc,it,client,attack_acc_mean,attack_acc_std,attack_pre_mean,attack_pre_std,attack_rec_mean,attack_rec_std
i0_c0.npy,0.894231,0.1260,0,c0,0.730502,0.010026,0.979139,0.019964,0.471154,0.014804
i0_c1.npy,0.849359,0.0925,0,c1,0.711538,0.007905,0.968068,0.012296,0.437500,0.012652
i0_c2.npy,0.826923,0.0930,0,c2,0.730235,0.010331,0.960243,0.011027,0.480235,0.015747
i0_c3.npy,0.875000,0.1152,0,c3,0.764156,0.018211,0.979512,0.025223,0.539530,0.026213
i0_c4.npy,0.903846,0.1034,0,c4,0.711271,0.012700,0.985418,0.022888,0.428953,0.019880
...,...,...,...,...,...,...,...,...,...,...
i14_c28.npy,0.983974,0.4599,14,c28,0.781250,0.006050,0.697611,0.005946,0.993056,0.001194
i14_c29.npy,0.996795,0.4164,14,c29,0.782853,0.009054,0.698648,0.008808,0.995192,0.003583
i14_c30.npy,1.000000,0.4548,14,c30,0.784989,0.009132,0.700938,0.009537,0.994658,0.003997
i14_c31.npy,0.990385,0.4438,14,c31,0.783921,0.005186,0.701893,0.004902,0.987179,0.004533


# MI LOAN

In [64]:
path = 'mia/loan/best_ams/'
aparams = {}
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if ".json" in file:
            with open(r+'/'+file) as json_file:
                aparams[r+file.replace('_params.json', '')] = json.load(json_file)
            
for r,d,f in os.walk(path):
    for file in f:
        if ".json" not in file and '.DS_Store' not in file:
            ams[r+file] = Net_attack(h_neurons=aparams[r+file]['h_neurons'], do=aparams[r+file]['do'], input_size=14)
            ams[r+file] = torch.load(r+'/'+file)      
        

In [65]:
path = 'loan/centr/'
tms_params = {}
tms = {}
for r,d,f in os.walk(path):
    for file in f:
        if ".json" in file:
            with open(r+'/'+file) as json_file:
                tms_params[file.replace('_params.json', '')] = json.load(json_file)
        if "target_model.npy" in file:
            tms[file.replace('.npy', '')] = np.load(path+'/'+file)
                    

In [66]:
# attack every centralized target model
for file in tms_params:
    
    if 'attack_acc_mean' in tms_params[file]:
        continue
    # set random seed and load the data    
    rand_seed=int(file[file.find('rs')+2:file.find('_lr')])
    x_target_train, y_target_train, x_target_test, y_target_test = scripts.load_loan(rand_seed, tr_size=10000)
    
    #attack
    target_model = algo.LogisticRegression_DPSGD()
    target_model.theta = tms[file]
    params = tms_params[file]
    scripts.set_model_params(target_model, params)
    attack_dict = attack.test_mi_attack(ams, target_model, x_target_train, y_target_train, x_target_test, y_target_test)
    
    params.update(attack_dict)
    if 'attack_acc' in params:
        params.pop('attack_acc')
        params.pop('attack_pre')
        params.pop('attack_rec')
    #write a new parameters file with attack results
    with open('loan/centr/'+file+'_params.json', 'w') as file:
        json.dump(params, file)
        

# Loan FL

In [67]:
path = 'loan/fl'
params = {}
results = {}
models = {}
for r,d,f in os.walk(path):
    for file in f:
        if "params.json" in file:
            with open(r+'/'+file) as json_file:
                params[r] = json.load(json_file)
        elif "results.csv" in file:
            results[r] = pd.read_csv(r+'/'+file)
            it = []
            client = []
            model_filenames = []
            if 'Unnamed: 0' in results[r].keys():
                for k in results[r]['Unnamed: 0']:
#                     print(k)
#                     break
                    it.append(k[k.find('i')+1:k.find('_')])
                    client.append(k[k.find('_')+1:])
                    model_filenames.append(k+'.npy')
                results[r]['it'] = it
                results[r]['client'] = client
                results[r]['file_name'] = model_filenames
                results[r].pop('Unnamed: 0')
        elif '.npy' in file:
            if r not in models:
                models[r] = {}
            models[r][file] = np.load(r+'/'+file)



In [68]:
path = 'mia/loan/best_ams/'
aparams = {}
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if ".json" in file:
            with open(r+'/'+file) as json_file:
                aparams[r+file.replace('_params.json', '')] = json.load(json_file)
            
for r,d,f in os.walk(path):
    for file in f:
        if ".json" not in file and '.DS_Store' not in file:
            ams[r+file] = Net_attack(h_neurons=aparams[r+file]['h_neurons'], do=aparams[r+file]['do'], input_size=14)
            ams[r+file] = torch.load(r+'/'+file)      
        

In [69]:
selected_files = results.keys()
len(selected_files)

322

In [70]:
# attack every federated local/global model
for file in selected_files:
    
    if 'attack_acc_mean' in results[file].keys():
        continue
        
    # set random seed and load the data    
    rand_seed=int(file[file.find('rs')+2:file.find('_ncl')])
    x_target_train, y_target_train, x_target_test, y_target_test = scripts.load_loan(rand_seed, tr_size=10000)
    #set number of client for following split of the training data between clients
    number_of_clients = len(results[file]['client'].unique())-1
    data_per_client = int(x_target_train.shape[0]/number_of_clients)

    #attack
    attack_results = {}
    print(file)
    for tm in results[file]['file_name']:
        target_model = algo.LogisticRegression_DPSGD()
        target_model.theta = models[file][tm]
        tm_params = params[file]
        scripts.set_model_params(target_model, tm_params)
        if 'g' in tm:
            target_model.x = x_target_train
            target_model.y = y_target_train
        else:
            i = int(tm[tm.find('_c')+2:tm.find('.npy')])
            target_model.x = x_target_train[i*data_per_client:(i+1)*data_per_client]
            target_model.y = y_target_train[i*data_per_client:(i+1)*data_per_client]
        attack_dict = attack.test_mi_attack(ams, target_model, target_model.x, target_model.y, x_target_test, y_target_test)
        attack_results[tm] = attack_dict
    attack_df = pd.DataFrame.from_dict(attack_results, orient='index')
    result_df = results[file].set_index('file_name')
    new_df = pd.merge(result_df, attack_df, left_index=True, right_index=True)
    #save attack results in old results file
    new_df.to_csv(file+'/results.csv')
    results[file] = new_df

loan/fl/rs42_ncl2_fiter10_lr0.01_iter200_reg0.0001_outDPlocalTrue_eps0.5
loan/fl/rs42_ncl4_fiter10_lr0.01_iter50_reg1e-06_sgdDPTrue_eps10000_L20_C2
loan/fl/rs42_ncl4_fiter10_lr0.01_iter100_reg0.0001_sgdDPTrue_eps10000_L20_C10
loan/fl/rs42_ncl2_fiter10_lr0.01_iter100_reg0.0001_outDPlocalTrue_eps5
loan/fl/rs42_ncl4_fiter10_lr0.01_iter50_reg1e-06_sgdDPTrue_eps10000_L20_C5
loan/fl/rs42_ncl2_fiter10_lr0.01_iter300_reg1e-06_outDPlocalTrue_eps100
loan/fl/rs42_ncl2_fiter10_lr0.01_iter100_reg0.0001_outDPlocalTrue_eps10000
loan/fl/rs42_ncl2_fiter10_lr0.01_iter200_reg0.0001_sgdDPTrue_eps10000_L20_C2
loan/fl/rs42_ncl2_fiter10_lr0.01_iter50_reg0.0001_sgdDPTrue_eps10000_L20_C8
loan/fl/rs42_ncl2_fiter10_lr0.01_iter100_reg1e-06_outDPlocalTrue_eps100000
loan/fl/rs42_ncl4_fiter10_lr0.01_iter200_reg1e-06_sgdDPTrue_eps10000_L20_C8
loan/fl/rs42_ncl2_fiter10_lr0.01_iter300_reg1e-06_outDPlocalTrue_eps1000
loan/fl/rs42_ncl2_fiter10_lr0.01_iter200_reg0.0001_sgdDPTrue_eps10000_L20_C5
loan/fl/rs42_ncl2_fiter10_l

loan/fl/rs42_ncl2_fiter10_lr0.01_iter200_reg1e-06_sgdDPTrue_eps10000_L1_C4
loan/fl/rs42_ncl4_fiter10_lr0.01_iter100_reg1e-06_sgdDPTrue_eps10000_L20_C2
loan/fl/rs42_ncl2_fiter10_lr0.01_iter200_reg1e-06_sgdDPTrue_eps10000_L5_C2
loan/fl/rs42_ncl2_fiter10_lr0.01_iter300_reg0.0001_sgdDPTrue_eps10000_L20_C1.5
loan/fl/rs42_ncl2_fiter10_lr0.01_iter200_reg1e-06_sgdDPTrue_eps10000_L5_C5
loan/fl/rs42_ncl2_fiter10_lr0.01_iter10_reg1e-06_outDPlocalTrue_eps50
loan/fl/rs42_ncl2_fiter10_lr0.01_iter200_reg1e-06_outDPlocalTrue_eps1000
loan/fl/rs42_ncl2_fiter10_lr0.01_iter100_reg1e-06_outDPlocalTrue_eps50000
loan/fl/rs42_ncl2_fiter10_lr0.01_iter200_reg0.0001_outDPlocalTrue_eps10
loan/fl/rs42_ncl2_fiter10_lr0.01_iter10_reg0.0001_outDPlocalTrue_eps10000
loan/fl/rs42_ncl2_fiter10_lr0.01_iter300_reg1e-06_outDPlocalTrue_eps10
loan/fl/rs42_ncl2_fiter10_lr0.01_iter300_reg1e-06_outDPlocalTrue_eps1
loan/fl/rs42_ncl2_fiter10_lr0.01_iter50_reg1e-06_outDPlocalTrue_eps0.5
loan/fl/rs42_ncl2_fiter10_lr0.01_iter10_reg1e

loan/fl/rs42_ncl2_fiter10_lr0.01_iter200_reg1e-06_sgdDPTrue_eps10000_L10_C1
loan/fl/rs42_ncl2_fiter10_lr0.01_iter10_reg0.0001_outDPlocalTrue_eps10
loan/fl/rs42_ncl4_fiter10_lr0.01_iter300_reg1e-06_sgdDPTrue_eps10000_L20_C5
loan/fl/rs42_ncl2_fiter10_lr0.01_iter200_reg1e-06_sgdDPTrue_eps10000_L20_C9
loan/fl/rs42_ncl2_fiter10_lr0.01_iter200_reg1e-06_sgdDPTrue_eps10000_L30_C1.5
loan/fl/rs42_ncl4_fiter10_lr0.01_iter50_reg0.0001_sgdDPTrue_eps10000_L20_C9
loan/fl/rs42_ncl2_fiter10_lr0.01_iter50_reg0.0001_outDPlocalTrue_eps5000
loan/fl/rs42_ncl4_fiter10_lr0.01_iter300_reg1e-06_sgdDPTrue_eps10000_L20_C2
loan/fl/rs42_ncl2_fiter10_lr0.01_iter200_reg1e-06_sgdDPTrue_eps10000_L10_C6
loan/fl/rs42_ncl4_fiter10_lr0.01_iter100_reg0.0001_sgdDPTrue_eps10000_L20_C8
loan/fl/rs42_ncl4_fiter10_lr0.01_iter300_reg0.0001_sgdDPTrue_eps10000_L20_C6
loan/fl/rs42_ncl4_fiter10_lr0.01_iter200_reg0.0001_sgdDPTrue_eps10000_L20_C8
loan/fl/rs42_ncl4_fiter10_lr0.01_iter100_reg0.0001_sgdDPTrue_eps10000_L20_C1.5
loan/fl/rs42